In [1]:
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline
import gc

import os
import string
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb

pd.options.mode.chained_assignment = None

## Importing Data

In [4]:
df_train = pd.read_csv(r"C:\Users\p_amb\Desktop\CapstoneFinal\DrugRec\www\Shuen's Data\kuc-hackathon-winter-2018\drugsComTrain_raw.csv", parse_dates=["date"])
df_test = pd.read_csv(r"C:\Users\p_amb\Desktop\CapstoneFinal\DrugRec\www\Shuen's Data\kuc-hackathon-winter-2018\drugsComTest_raw.csv", parse_dates=["date"])

In [5]:
df_all = pd.concat([df_train,df_test])

In [6]:
df_disease = pd.read_csv(r"C:\Users\p_amb\Desktop\CapstoneFinal\DrugRec\www\Shuen's Data\sdsort\dia_t.csv")
df_symptoms = pd.read_csv(r"C:\Users\p_amb\Desktop\CapstoneFinal\DrugRec\www\Shuen's Data\sdsort\sym_t.csv")
df_dis_sym = pd.read_csv(r"C:\Users\p_amb\Desktop\CapstoneFinal\DrugRec\www\Shuen's Data\sdsort\diffsydiw.csv")

In [7]:
df_dis_sym = df_dis_sym[['did', 'syd', 'wei']]
df_dis_sym = df_dis_sym.sort_values(axis=0, by=['did'])
df_dis_sym = df_dis_sym.reset_index(drop=True)
df_dis_sym = df_dis_sym.rename(columns={"did": "disease", "syd": "symptom", "wei": "weight"})
df_dis_sym = df_dis_sym.drop(axis=0, index=5568)
df_dis_sym.head()

,disease,symptom,weight
0,1.0,7.0,1.0
1,1.0,275.0,1.0
2,1.0,173.0,1.0
3,1.0,54.0,1.0
4,1.0,32.0,1.0


In [8]:
df_disease.head()

,did,diagnose
0,1,Abdominal aortic aneurysm (enlarged major bloo...
1,2,Abdominal swelling
2,3,Abdominal trauma
3,4,Abrasions (scrapes)
4,5,ACE inhibitor induced cough blood pressure med...


In [9]:
yea = list(df_disease.iloc[:,0])
len(yea)

1166

## Defining function that returns match percent of disease given symptoms

In [10]:
disease_list = []
for item in df_dis_sym.iloc[:, 0]:
    if item not in disease_list:
        disease_list.append(item)
len(disease_list)

1166

In [11]:
#creating new dataframe, condensed version of df_dis_sym
df_dis_sym_c = pd.DataFrame(columns={'disease', 'symptoms'})
sym_lst = []
for row in range(len(df_dis_sym)):
    if row == 0:
        sym_lst.append(df_dis_sym.iloc[row, 1])
    elif row == len(df_dis_sym)-1:
        sym_lst = sorted(sym_lst)
        df_dis_sym_c = df_dis_sym_c.append({'disease': df_dis_sym.iloc[row, 0], 'symptoms': sym_lst}, ignore_index=True)
    elif (df_dis_sym.iloc[row, 0] == df_dis_sym.iloc[row-1, 0]):
        sym_lst.append(df_dis_sym.iloc[row, 1])
    elif (df_dis_sym.iloc[row, 0] != df_dis_sym.iloc[row-1, 0]):
        sym_lst = sorted(sym_lst)
        df_dis_sym_c = df_dis_sym_c.append({'disease': df_dis_sym.iloc[row-1, 0], 'symptoms': sym_lst}, ignore_index=True)
        sym_lst = [df_dis_sym.iloc[row, 1]]
columnsTitles=["disease","symptoms"]
df_dis_sym_c=df_dis_sym_c.reindex(columns=columnsTitles)
df_dis_sym_c.head()
    

,disease,symptoms
0,1.0,"[7.0, 32.0, 54.0, 173.0, 205.0, 275.0]"
1,2.0,"[99.0, 284.0]"
2,3.0,"[2.0, 102.0]"
3,4.0,"[6.0, 11.0, 13.0, 23.0, 39.0, 41.0, 57.0, 67.0..."
4,5.0,"[17.0, 126.0, 159.0, 194.0, 195.0]"


In [12]:
# df_symptoms.head()
df_disease.head()

,did,diagnose
0,1,Abdominal aortic aneurysm (enlarged major bloo...
1,2,Abdominal swelling
2,3,Abdominal trauma
3,4,Abrasions (scrapes)
4,5,ACE inhibitor induced cough blood pressure med...


In [13]:
#creating a dictionary of values with the keys being the syd indexes and the values being the symptom (index to symptom)
symp_dict = {}
for i in range(len(df_symptoms)):
    symp_dict[df_symptoms.iloc[i, 0]] = df_symptoms.iloc[i, 1]

In [14]:
#creating opposite dictionary (symptom to index)
symp_dict2 = {}
for i in range(len(df_symptoms)):
    symp_dict2[df_symptoms.iloc[i, 1]] = df_symptoms.iloc[i, 0]

In [15]:
#creating a dictionary of values with the keys being the did indexes and the values being the disease (index to disease)
dis_dict = {}
for i in range(len(df_disease)):
    dis_dict[df_disease.iloc[i, 0]] = df_disease.iloc[i, 1]

In [16]:
#creating opposite dictionary (disease to index)
dis_dict2 = {}
for i in range(len(df_disease)):
    dis_dict2[df_disease.iloc[i, 1]] = df_disease.iloc[i, 0]

In [17]:
#returns the top five diseases with matching symptoms, if ties then returns all
def return_disease(*symptoms):
    top_five = []
    value_lst = []
    per_match_dic = {}
    for row in range(len(df_dis_sym_c)):
        match = 0
        total = 0
        for symptom in symptoms:
            lst = df_dis_sym_c.iloc[row, 1]
            if symp_dict2[symptom] in lst:
                match += 1
                total += 1
            else:
                total += 1
        per_match = match/total
        disease_index = df_dis_sym_c.iloc[row, 0]
        disease = dis_dict[disease_index]
        per_match_dic[disease] = per_match
    for value in per_match_dic.values():
        value_lst.append(value)
    value_lst = sorted(value_lst)
    value_lst_top = value_lst[-1:-6:-1]
    for disease, value in per_match_dic.items():
        if value in value_lst_top:
            top_five.append((disease, value))
    return top_five

In [18]:
return_disease('Upper abdominal pain', 'Lower abdominal pain', 'Alcohol abuse')

[('Alcoholism', 0.6666666666666666),
 ('Appendicitis', 0.6666666666666666),
 ('Celiac disease\x0bceliac sprue', 0.6666666666666666),
 ('Cholecystitis\x0binflammation of the gallbladder', 0.6666666666666666),
 ('Cholelithiasis\x0bgallstones', 0.6666666666666666),
 ('Colonic diverticulitis\x0binflammation of pouches in the large intestine',
  0.6666666666666666),
 ('Constipation', 0.6666666666666666),
 ('Endometriosis\x0buterine tissue outside the uterus', 0.6666666666666666),
 ('Gastric ulcer\x0bstomach ulcer', 0.6666666666666666),
 ('Gastroenteritis\x0bintestinal infection', 0.6666666666666666),
 ('Gastroesophageal reflux\x0bGERD, heartburn', 0.6666666666666666),
 ('Hernia', 0.6666666666666666),
 ('Inguinal hernia\x0bgroin hernia', 0.6666666666666666),
 ('Irritable bowel syndrome\x0bIBS, chronic intestinal pain and cramping',
  0.6666666666666666),
 ('Ligament sprain\x0bconnection between two bones', 0.6666666666666666),
 ('Kidney stone\x0bnephrolithiasis', 0.6666666666666666),
 ('Ovar

In [19]:
#returns the top five diseases with matching symptoms, if ties then returns all, also return id
def return_disease2(*symptoms):
    top_five = []
    value_lst = []
    per_match_dic = {}
    for row in range(len(df_dis_sym_c)):
        match = 0
        total = 0
        for symptom in symptoms:
            lst = df_dis_sym_c.iloc[row, 1]
            if symp_dict2[symptom] in lst:
                match += 1
                total += 1
            else:
                total += 1
        per_match = match/total
        disease_index = df_dis_sym_c.iloc[row, 0]
        disease = dis_dict[disease_index]
        per_match_dic[(disease, disease_index)] = per_match
    for value in per_match_dic.values():
        value_lst.append(value)
    value_lst = sorted(value_lst)
    value_lst_top = value_lst[-1:-6:-1]
    for (disease, index), value in per_match_dic.items():
        if value in value_lst_top:
            top_five.append(((disease, index), value))
    return top_five

In [41]:
result = return_disease2('Upper abdominal pain', 'Lower abdominal pain', 'Alcohol abuse')

In [40]:
print(x)

[(('Alcoholism', 25.0), 0.6666666666666666), (('Appendicitis', 56.0), 0.6666666666666666), (('Celiac disease\x0bceliac sprue', 139.0), 0.6666666666666666), (('Cholecystitis\x0binflammation of the gallbladder', 163.0), 0.6666666666666666), (('Cholelithiasis\x0bgallstones', 165.0), 0.6666666666666666), (('Colonic diverticulitis\x0binflammation of pouches in the large intestine', 179.0), 0.6666666666666666), (('Constipation', 187.0), 0.6666666666666666), (('Endometriosis\x0buterine tissue outside the uterus', 241.0), 0.6666666666666666), (('Gastric ulcer\x0bstomach ulcer', 306.0), 0.6666666666666666), (('Gastroenteritis\x0bintestinal infection', 308.0), 0.6666666666666666), (('Gastroesophageal reflux\x0bGERD, heartburn', 309.0), 0.6666666666666666), (('Hernia', 349.0), 0.6666666666666666), (('Inguinal hernia\x0bgroin hernia', 388.0), 0.6666666666666666), (('Irritable bowel syndrome\x0bIBS, chronic intestinal pain and cramping', 401.0), 0.6666666666666666), (('Ligament sprain\x0bconnection

In [22]:
## Defining function that returns similiar condition in the new dataset

In [23]:
def getCondition(str):
    out = df
    out = df[df['Condition'] == str]
    out =out.reset_index(drop = True) 
    return out

In [187]:
df = pd.read_csv(r"C:\Users\p_amb\Desktop\Data\processed.csv")
usefulratio = []

from collections import Counter 

Counter = Counter(df['condition'])

commonDrugs = Counter.most_common()


df = df.dropna()
df.head()




,Unnamed: 0,drugName,condition,review,cleanReview,rating,ratingSentiment,ratingSentimentLabel,vaderReviewScore,vaderSentiment,vaderSentimentLabel,aggregateRating
0,0,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...","""I&#039;ve tried antidepressants years (citalo...",10.0,2,positive,0.7623,2,positive,100.0
1,1,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...","""My son Crohn&#039;s disease done well Asacol....",8.0,2,positive,0.4767,2,positive,90.0
2,2,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""","""Quick reduction symptoms""",9.0,2,positive,0.0000,0,neutral,95.0
3,3,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...","""Contrave combines drugs used alcohol, smoking...",9.0,2,positive,0.8115,2,positive,95.0
4,4,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...","""I birth control one cycle. After reading revi...",9.0,2,positive,0.9619,2,positive,95.0


In [188]:
druglist=[]
for x in commonDrugs:
    druglist.append(x[0])
    
    
    
    
print(druglist)

['Birth Control', 'Depression', 'Pain', 'Anxiety', 'Acne', 'Bipolar Disorde', 'Insomnia', 'Weight Loss', 'Obesity', 'ADHD', 'Diabetes, Type 2', 'Emergency Contraception', 'High Blood Pressure', 'Vaginal Yeast Infection', 'Abnormal Uterine Bleeding', 'Bowel Preparation', 'Smoking Cessation', 'ibromyalgia', 'Migraine', 'Anxiety and Stress', 'Major Depressive Disorde', 'Constipation', 'Chronic Pain', 'Panic Disorde', 'Migraine Prevention', 'Urinary Tract Infection', 'Muscle Spasm', 'Osteoarthritis', 'Generalized Anxiety Disorde', 'Opiate Dependence', 'Erectile Dysfunction', 'Irritable Bowel Syndrome', 'Allergic Rhinitis', 'Rheumatoid Arthritis', 'Bacterial Infection', 'Cough', nan, 'Sinusitis', 'Nausea/Vomiting', 'GERD', 'Hyperhidrosis', 'Overactive Bladde', 'Multiple Sclerosis', 'Hepatitis C', 'HIV Infection', 'High Cholesterol', 'Back Pain', 'Restless Legs Syndrome', 'Psoriasis', 'Schizophrenia', 'Constipation, Chronic', 'Obsessive Compulsive Disorde', 'Endometriosis', 'Benign Prostatic

In [191]:


#print(druglist[35])

cleanedList = [i for i in druglist if type(i) is not float]

#print(fuzz.partial_ratio(result[0][0][0],cleanedList[36]))
#result

val = []
finallist = []
finallistindex  = []
for x in range(0,len(result)):
    
    olddis = result[x][0][0]
    val = []
    for y in range(0,len(cleanedList)):
        
        #print('num:' + y)
        #print(isinstance(druglist[y],)) 
        val.append(fuzz.partial_ratio(olddis,cleanedList[y]))
    values = np.array(val)
    searchval = max(val)
    ii = np.where(values == searchval)[0]
    for i in ii:
        print(cleanedList[i])
        finallist.append([cleanedList[i],val[i],x])
    print('..............................')    
    
        

Alcohol Withdrawal
..............................
Meningitis
..............................
Gas
mis
..............................
Cholera
..............................
Cholera
..............................
Diverticulitis
..............................
Constipation
Constipation, Chronic
Constipation, Drug Induced
Chronic Idiopathic Constipation
Opioid-Induced Constipation
Constipation, Acute
..............................
Endometriosis
me
..............................
Gas
..............................
Gastroenteritis
Gas
..............................
GERD
Gas
..............................
Schizophrenia
Otitis Externa
Acute Otitis Externa
Thrombocytopenia
..............................
Angina
..............................
me
..............................
me
..............................
Ehrlichiosis
..............................
Ovarian Cysts
..............................
Cance
..............................
Cance
..............................
Bone infection
................

In [192]:
maxval = 0;
tempmax = []
for i in range finallist
    x = finalllist[i][1]

finallist 

[['Alcohol Withdrawal', 80, 0],
 ['Meningitis', 74, 1],
 ['Gas', 67, 2],
 ['mis', 67, 2],
 ['Cholera', 71, 3],
 ['Cholera', 71, 4],
 ['Diverticulitis', 93, 5],
 ['Constipation', 100, 6],
 ['Constipation, Chronic', 100, 6],
 ['Constipation, Drug Induced', 100, 6],
 ['Chronic Idiopathic Constipation', 100, 6],
 ['Opioid-Induced Constipation', 100, 6],
 ['Constipation, Acute', 100, 6],
 ['Endometriosis', 100, 7],
 ['me', 100, 7],
 ['Gas', 100, 8],
 ['Gastroenteritis', 100, 9],
 ['Gas', 100, 9],
 ['GERD', 100, 10],
 ['Gas', 100, 10],
 ['Schizophrenia', 73, 11],
 ['Otitis Externa', 73, 11],
 ['Acute Otitis Externa', 73, 11],
 ['Thrombocytopenia', 73, 11],
 ['Angina', 83, 12],
 ['me', 100, 13],
 ['me', 100, 14],
 ['Ehrlichiosis', 75, 15],
 ['Ovarian Cysts', 85, 16],
 ['Cance', 80, 17],
 ['Cance', 80, 18],
 ['Bone infection', 86, 19],
 ['Bladder Infection', 94, 20],
 ['mis', 67, 21],
 ['Lactose Intolerance', 95, 22],
 ['Diverticulitis', 86, 23],
 ['min)', 75, 24]]

In [185]:
for x in range(0,len(result)):
    print( result[x][0][0])

Alcoholism
Appendicitis
Celiac diseaseceliac sprue
Cholecystitisinflammation of the gallbladder
Cholelithiasisgallstones
Colonic diverticulitisinflammation of pouches in the large intestine
Constipation
Endometriosisuterine tissue outside the uterus
Gastric ulcerstomach ulcer
Gastroenteritisintestinal infection
Gastroesophageal refluxGERD, heartburn
Hernia
Inguinal herniagroin hernia
Irritable bowel syndromeIBS, chronic intestinal pain and cramping
Ligament sprainconnection between two bones
Kidney stonenephrolithiasis
Ovarian cystfemale organ fluid collection
Pancreatitispancreas inflammation
Pancreas inflammationpancreatitis, chronic
Pyelonephritiskidney infection
Bladder infectioncystitis, UTI, urinary tract infection
Umbilical herniabelly button hernia
Lactose intolerance
Diverticulosisweakening of the large intestine wall
Ventral herniabulging of the abdominal wall


In [52]:
condition = df['condition']

In [68]:
commonDrugs



['Birth Control', 'Depression', 'Pain', 'Anxiety', 'Acne', 'Bipolar Disorde', 'Weight Loss', 'Insomnia', 'Obesity', 'ADHD', 'Emergency Contraception', 'Vaginal Yeast Infection', 'Diabetes, Type 2', 'High Blood Pressure', 'Smoking Cessation', 'Abnormal Uterine Bleeding', 'Bowel Preparation', 'Migraine', 'ibromyalgia', 'Anxiety and Stress', 'Constipation', 'Major Depressive Disorde', 'Chronic Pain', 'Panic Disorde', 'Migraine Prevention', 'Urinary Tract Infection', 'Opiate Dependence', 'Osteoarthritis', 'Muscle Spasm', 'Erectile Dysfunction', 'Generalized Anxiety Disorde', 'Allergic Rhinitis', 'Irritable Bowel Syndrome', 'Rheumatoid Arthritis', 'Bacterial Infection', 'Cough', nan, 'Sinusitis', 'Nausea/Vomiting', 'GERD', 'Hepatitis C', 'Hyperhidrosis', 'Restless Legs Syndrome', 'Overactive Bladde', 'Multiple Sclerosis', 'High Cholesterol', 'HIV Infection', 'Psoriasis', 'Schizophrenia', 'Hypogonadism, Male', 'Endometriosis', 'Back Pain', 'Constipation, Chronic', 'Seizures', 'Rosacea', 'Obs

In [127]:
df_disease['diagnose']
from collections import Counter 

Counter = Counter(df_disease['diagnose'])

commonDis = Counter.most_common()
commonDis1 =[]

commonDis[0][0]

for x in range(0,len(commonDis)):
    #print(commonDis[x][0])
    commonDis1.append(commonDis[x][0])

commonDis2=[]   
    
for x in commonDis1:
    y =  x.replace("\x0b"," ")
    z= y.replace("\x92 "," ")
    commonDis2.append(z)
    
print(commonDis2)

['Abdominal aortic aneurysm (enlarged major blood vessel)', 'Abdominal swelling', 'Abdominal trauma', 'Abrasions  (scrapes)', 'ACE inhibitor induced cough blood pressure medication side effect Drug-induced cough Drug reaction: ACE inhibitor', 'acetaminophen overdose Adverse reaction to acetaminophen', 'Tylenol  acetaminophen poisoning', 'Achilles tendonitis  (heel tendon inflammation)', 'Achilles tendon rupture (heel tendon tear)', 'Acid  LSD abuse', 'Acidosis  (excessive acid in the body)', 'Acoustic neuroma (ear nerve tumor)', 'AC joint separation acromioclavicular joint separation closed separation of acromioclavicular joint', 'Acute angle closure glaucoma increased inner eye pressure', 'Acute fatty liver of pregnancy', 'Adenoiditis (a type of lymph node inflammation)', 'Adenovirus infection  (virus infection)', 'Frozen shoulder  (adhesive capsulitis of shoulder)', 'Adjustment disorder  (poor adjustment to life stressor)', 'Alcohol  ethanol intoxication', 'Alcohol  ethanol abuse', '

In [118]:
commonDis1[0].replace("\x0b"," ")

'Abdominal aortic aneurysm (enlarged major blood vessel)'